<a href="https://colab.research.google.com/github/gmihaila/machine_learning_toolbox/blob/master/universal_sentence_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Creating universal sentence embedding

### Make tf hub safe to specific folder
export TFHUB_CACHE_DIR=/usr/local/my_module_cache

[TF Module](https://www.tensorflow.org/hub/modules/google/universal-sentence-encoder-large/3)

In [0]:
!pip install gensim


In [0]:
# Download Google Word2Vec
print("\n\nDownloading...")
!wget https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
# Unzip 
print("\n\nUnzip...")
!gunzip GoogleNews-vectors-negative300.bin.gz
# Check File
print("Finsihed!")
!ls

In [0]:
print("\n\nDownload ZIP...")
!wget https://www.dropbox.com/s/6ve2ssmxltagcuy/tf_sentence_emb.zip
print("\n\nUnzip...")
!unzip tf_sentence_emb.zip
print("\n\nFinished!")
!ls


In [0]:
import os
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from keras import backend as K
from keras import layers
from keras.models import Model
from gensim.models import KeyedVectors



# Onlye used for testing
def CosineSim(a, b):
    """Takes 2 vectors a, b and returns the cosine similarity according 
      to the definition of the dot product
    """
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)

    return dot_product / (norm_a * norm_b)

In [0]:
sentence_1 = "I am hungry"
sentence_2 = "I am starving"

In [0]:
path_emb = "GoogleNews-vectors-negative300.bin"

print('Loading pre-trained embedding model...')
word_emb = KeyedVectors.load_word2vec_format(path_emb, binary=True)
print('Finished!\n')

In [0]:
'''
  Deep Averaging Network Fucntion
'''

def Dan(text, emb_size, emb_model):
    emb_dan = np.zeros(emb_size, dtype=float)
    num_words = 0
    for word in text.split():
        if word in emb_model.vocab:
            # add each embedding
            emb_dan += np.array(emb_model[word], dtype=float)
            num_words += 1
    # average embedding
    emb_dan = emb_dan/num_words
    return emb_dan

In [0]:
sentence_emb_1 = Dan(sentence_1, emb_size=300, emb_model=word_emb)
sentence_emb_2 = Dan(sentence_2, emb_size=300, emb_model=word_emb)

CosineSim(sentence_emb_1, sentence_emb_2)

In [0]:
path_hub = "tf_sentence_emb"


# keras model
def UniversalEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)), 
                 signature="default", as_dict=True)["default"]

# load pre-trained model
embed = hub.Module(path_hub)

# input layer
input_text = layers.Input(shape=(1,), dtype=tf.string)

# embedding layer output
embedding = layers.Lambda(UniversalEmbedding,
                          output_shape=(512,))(input_text)

'''
    Add here rest of your model if needed
'''

# assamble model
model = Model(inputs=[input_text], outputs=embedding)

# compile model
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

# show modle architecture
print(model.summary())

In [0]:
emb_txt = None
in_txt = np.array([sentence_1, sentence_2])
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    emb_txt = model.predict(in_txt)
    
sentence_emb_1 = emb_txt[0]
sentence_emb_2 = emb_txt[1]

CosineSim(sentence_emb_1, sentence_emb_2)